<div align="left">
<img src="https://licensebuttons.net/l/by-nc-sa/3.0/88x31.png" align="right">
<p align="left"><b>Authors: Mikel Baraza Vidal, Daniele Borghesi, Eva Cantín Larumbe, Francesco Pio Capoccello, Eva Teisberg</b></p>
</div>
<img align="left" width="30%" src="https://www.inf.upv.es/www/etsinf/wp-content/uploads/2017/02/ETSInf_PRINCIPAL_V-horizontal.png"/> <img width="30%" src="https://www.upv.es/perfiles/pas-pdi/imagenes/marca_UPV_principal_negro150.png" align="right"/>


In [1]:
#import folium
import requests
import json
#import geopandas as geo
from io import StringIO
import io
import pandas as pd
#------------------------------------

from ortools.linear_solver import pywraplp
from numpy import average, std
from statistics import *
import folium
import numpy as np

###  Datasets

In [2]:
vehicle_types = ["engine", "ladder", "rescue", "squad", "hazardous"]

In [3]:
serviceability_distances = {
    "engine": 800.24,
    "ladder": 800.24,
    "rescue": 2675.74,
    "squad": 2339.26,
    "hazardous": 4537.84,
}

In [4]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones.json")
distances_stations_db = json.loads(response.text)

df_distances_stations = pd.DataFrame(distances_stations_db)
df_distances_stations =  pd.concat([df_distances_stations.iloc[-1:], df_distances_stations.iloc[:-1]], axis=0)
df_distances_stations.fillna(0, inplace=True)
df_distances_stations

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
42 South Street,0.00,228.56,282.07,367.00,299.82,397.01,523.64,609.91,444.76,511.11,...,1384.71,1348.97,1337.57,1553.31,2392.53,2406.78,2697.62,2833.72,2500.30,1567.00
49 Beekman Street,154.94,0.00,175.70,260.63,261.52,391.57,494.09,549.25,330.31,403.98,...,1401.76,1366.02,1354.62,1570.37,2409.58,2423.84,2609.88,2719.26,2396.89,1584.06
100 Duane Street,299.43,180.79,0.00,96.76,272.78,413.52,467.01,447.83,228.89,302.56,...,1411.62,1375.88,1364.48,1580.22,2419.44,2341.17,2469.12,2578.51,2256.14,1593.91
14 N. Moore Street,408.12,289.48,174.89,0.00,339.69,444.26,450.79,431.61,212.68,286.35,...,1520.31,1484.57,1473.17,1688.91,2405.59,2246.56,2374.51,2483.90,2161.52,1702.60
75 Canal Street,296.75,295.70,275.65,283.51,0.00,195.55,260.18,346.45,260.93,293.16,...,1563.49,1522.61,1516.35,1732.10,2571.31,2470.82,2598.77,2708.16,2385.79,1745.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345 Annadale Road,2387.88,2470.02,2355.43,2311.84,2446.72,2547.14,2553.66,2534.49,2315.55,2389.22,...,1315.83,1303.25,1115.74,877.46,478.78,0.00,300.33,537.43,414.36,1461.28
1560 Drumgoole Road West,2566.73,2569.97,2455.37,2411.78,2546.66,2647.08,2653.60,2634.43,2415.49,2489.16,...,1494.67,1482.10,1294.59,1056.30,560.66,220.62,0.00,411.10,284.85,1640.12
7219 Amboy Road,2891.38,2772.75,2658.16,2614.56,2749.45,2849.86,2856.39,2837.21,2618.27,2691.95,...,1847.28,1834.71,1647.20,1408.91,898.57,580.71,391.45,0.00,521.33,1992.73
1100 Rossville Ave,2425.57,2306.93,2192.34,2148.75,2283.63,2384.04,2390.57,2371.40,2152.46,2226.13,...,1408.77,1466.73,1208.69,1182.77,763.62,423.58,255.42,486.65,0.00,1554.22


In [5]:
distance = pd.read_csv("dataset/distance_stations_neighborhoods.csv", index_col=0)
neighborhood = pd.read_csv("dataset/neighborhoods_info_standardized.csv")
serviceability = {}

for vehicle_type in vehicle_types:
    serviceability[vehicle_type] = pd.read_csv("dataset/serviceability/" + vehicle_type +"_ntas_serviceability.csv", index_col=0)

### Creating an instance of the Solver class

In [6]:
solver = pywraplp.Solver.CreateSolver("SAT")

### Creating decision variables

with:

$X_{i, j, k}: $  number of vehicles of type ${i}$ deployed in station ${j}$ during shift ${k}$

$X_{i, j, k} \geq 0$, $X_{i, j, k} \in \mathbb{Z}$

$ i \in I, j \in J, k \in K$

_________________

$Y_{i, a, b}:$  number of vehicles of type ${i}$ moved from station ${a}$ to station ${b}$ at shift change

$Y_{i, a, b} \geq 0$, $Y_{i, a, b} \in \mathbb{Z}$

$ i \in I, a \in J, b \in J$

_____

$T_{i, j, k}: 1$ if a vehicle of type ${i}$ is deployed in station ${j}$ during shift ${k}$, $0$ otherwise

$T_{i, j, k} \geq 0$, $T_{i, j, k} \in  \{0,1\}$

$ i \in I, j \in J, k \in K$

____


$Q_{j, k}: 1$ if there are more than two vehicles in station $j$ during shift $k$, $0$ otherwise

$Q_{j, k} \geq 0$, $Q_{j, k} \in \{0,1\}$

$ j \in J, k \in K$

____

- type of vehicles$: i=1\dots 5$

- stations: $j=1\dots 219$

- shifts: $k=1\dots 2$

- neighborhoods: $n=1 \dots 195$


In [7]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents = json.loads(response.text)

I = ["engine", "ladder", "rescue", "squad", "hazardous"]

In [8]:
N = list(neighborhood.nta)
J = list(distance.columns)
K = [1,2]

In [9]:
X = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.IntVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " deployed at station "+ str(j)+ " during shift " + str(k))
            X[(i,j,k)] = v

In [10]:
Y = {}
for i in I:
    for a in J:
        for b in J:
            v = solver.IntVar(0, solver.infinity(), "Number of vehicles of type " + str(i) + " moved from station "+ str(a)+ " to station " + str(b))
            Y[(i,a,b)] = v

In [11]:
T = {}
for i in I:
    for j in J:
        for k in K:
            v = solver.BoolVar("if a vehicle of type" + str(i)  + " is deployed in station " + str(j) + " during shift " + str(k))
            T[(i,j,k)] = v

In [12]:
Q = {}
for j in J:
    for k in K:
        v = solver.BoolVar("if there are two or more vehicles in station " + str(j) + " during shift " + str(k))
        Q[(j,k)] = v

## Creting constraints

### 1. Station capacity

\begin{equation}
    \sum_{i \in I}^{}X_{i, j, k} \leq c_{j}, \forall j \in J, k \in K
    \label{station_capacity}
\end{equation}

In [13]:
response2 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/located_firehouses.json")
stations = json.loads(response2.text)

In [14]:
#capacities = [(feature['properties']['capacity'], feature['properties']['FacilityAddress']) for feature in stations['features']]

capacities = {}

for station in stations["features"]:
    station_address = station["properties"]["FacilityAddress"]
    capacities[station_address] = station["properties"]["capacity"]

In [15]:
station_capacity_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(-solver.infinity(), capacity)
    
        for i in I:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)

        station_capacity_constraints.append(c)

assert len(station_capacity_constraints) == 438, "The number of constraints (" + str(len(station_capacity_constraints)) + ") is NOT correct"

### 2. Vehicles availability

\begin{equation}
    \sum_{j \in J}^{}X_{i, j, k} = v_{i}, \forall i \in I, k \in K
    \label{vehicle_availability}
\end{equation}

In [16]:
availabilities = {'engine': 197, 'ladder': 143, 'hazardous': 1, 'squad': 8, 'rescue':5}

In [17]:
vehicles_availability_constraints = []

for i in I:
    availability = availabilities[i]

    for k in K:
        c = solver.Constraint(availability, availability)

        for j in J:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
            
        vehicles_availability_constraints.append(c)

assert len(vehicles_availability_constraints) == 10, "The number of constraints (" + str(len(vehicles_availability_constraints)) + ") is NOT correct"

### 3. At least one vehicle

\begin{equation}
    \sum_{i \in I}^{}X_{i, j, k} \geq 1, \forall j \in J, k \in K
    \label{at_least_one_vehicle}
\end{equation}

In [18]:
atleast_one_vehicle_constraints = []

for j in J:
    for k in K:
        c = solver.Constraint(1, solver.infinity())
    
        for i in I:
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
        atleast_one_vehicle_constraints.append(c)

assert len(atleast_one_vehicle_constraints) == 438, "The number of constraints (" + str(len(atleast_one_vehicle_constraints)) + ") is NOT correct"

### 4. Scarce resources coverage

\begin{equation}
    \sum_{j \in J}^{}s_{i, j, n}X_{i, j, k} \geq  1, \forall n \in N, k \in K, i \in \left \{ 3, 4 \right \}
    \label{scarce_resources}
\end{equation}

In [19]:
scarce_resources_availability = []

for k in K:
    for n in N:
        for i in ["rescue", "squad"]:
            c = solver.Constraint(1, solver.infinity())
            
            for j in J:
                s = 1 if serviceability[i].loc[n, j] == True else 0
                v = X[(i, j, k)]
                c.SetCoefficient(v, s)

            scarce_resources_availability.append(c)

assert len(scarce_resources_availability) == 780, "The number of constraints (" + str(len(scarce_resources_availability)) + ") is NOT correct"

### 5. Area resources concentration

\begin{equation}
    \theta = \sum_{j \in J}^{}s_{i, j, n}
\end{equation}

\begin{equation}
    \frac{\theta}{2}\leq \sum_{j \in J}^{}s_{i, j, n}X_{i, j, k} \leq \theta, \forall n \in N, k \in K, i \in \left \{ 1, 2 \right \}
    \label{engine_ladder_concentration_area}
\end{equation}

\begin{equation}
    r_{a, b} + \frac{l_{i}}{2} \geq \frac{l_{i}}{2} \left (T_{i, a, k} + T_{i, b, k} \right ), \forall k \in K, i \in \left \{ 3, 4 \right \}
    \label{rescue_squad_concentration_area}
\end{equation}

In [20]:
area_resources_concentration_constraints = []

for k in K:
    for n in N:
        for i in ["engine", "ladder"]:
            s_max = 0
            
            for j in J:
                s_max += 1 if serviceability[i].loc[n, j] == True else 0
            
            c = solver.Constraint(s_max / 2, s_max)
            
            for j in J:
                s = 1 if serviceability[i].loc[n, j] == True else 0
                v = X[(i, j, k)]
                c.SetCoefficient(v, s)

            area_resources_concentration_constraints.append(c)
        
assert len(area_resources_concentration_constraints) == 780, "The number of constraints (" + str(len(area_resources_concentration_constraints)) + ") is NOT correct"

In [21]:
scarce_resources_concentration = []

for k in K:
    for a in J:
        for b in J:
            route_distance = df_distances_stations.loc[a, b]
            for i in ["rescue", "squad"]:
                type_distance_limit = serviceability_distances[i] / 2
                
                c = solver.Constraint(-solver.infinity(), route_distance + type_distance_limit)
            
                T_a = T[(i, a, k)]
                T_b = T[(i, b, k)]

                c.SetCoefficient(T_a, type_distance_limit)
                c.SetCoefficient(T_b, type_distance_limit)

                scarce_resources_concentration.append(c)

assert len(scarce_resources_concentration) == 191844, "The number of constraints (" + str(len(scarce_resources_concentration)) + ") is NOT correct"

### 6. Station resources concentration

\begin{equation}
    X_{i, j, k} \leq  2, \forall j \in J, k \in K, i \in \left \{ 1, 2 \right \}
    \label{engine_ladder_concentration_station}
\end{equation}

\begin{equation}
    X_{i, j, k} \leq  1, \forall j \in J, k \in K, i \in \left \{ 3, 4 \right \}
    \label{rescue_squad_concentration_station}
\end{equation}

In [22]:
station_resources_concentration_constraints = []

for k in K:
    for j in J:
        for i in ["engine", "ladder"]:
            c = solver.Constraint(-solver.infinity(), 2)
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
            station_resources_concentration_constraints.append(c)
                    
assert len(station_resources_concentration_constraints) == 876, "The number of constraints (" + str(len(station_resources_concentration_constraints)) + ") is NOT correct"

In [23]:
station_resources_concentration_constraints = []

for k in K:
    for j in J:
        for i in ["rescue", "squad"]:
            c = solver.Constraint(-solver.infinity(), 1)
            v = X[(i, j, k)]
            c.SetCoefficient(v, 1)
            station_resources_concentration_constraints.append(c)
                    
assert len(station_resources_concentration_constraints) == 876, "The number of constraints (" + str(len(station_resources_concentration_constraints)) + ") is NOT correct"

### 7. At least two vehicle types

#### 7.1 Linking constrain T

\begin{equation}
    T_{i, j, k} \leq X_{i, j, k} \leq c_{j}T_{i, j, k}, \forall i \in I, j \in J, k \in K
    \label{linking_T}
\end{equation}

In [24]:
T_lower_linking_constraints = []
T_upper_linking_constraints = []

for i in I:
    for j in J:
        capacity = capacities[j]
        for k in K:
            v_T = T[(i, j, k)]
            v_X = X[(i, j, k)]

            c_lower = solver.Constraint(-solver.infinity(), 0)

            c_lower.SetCoefficient(v_T, 1)
            c_lower.SetCoefficient(v_X, -1)

            T_lower_linking_constraints.append(c_lower)

            c_upper = solver.Constraint(-solver.infinity(), 0)

            c_upper.SetCoefficient(v_T, -capacity)
            c_upper.SetCoefficient(v_X, 1)

            T_upper_linking_constraints.append(c_upper)

assert len(T_lower_linking_constraints) == 2190, "The number of constraints (" + str(len(T_lower_linking_constraints)) + ") is NOT correct"
assert len(T_upper_linking_constraints) == 2190, "The number of constraints (" + str(len(T_upper_linking_constraints)) + ") is NOT correct"

#### 7.2. Linking constraint Q


\begin{equation}
    \sum_{i \in I} X_{i, j, k} - 1 \leq c_{j}Q_{j, k}, \forall j \in J, k \in K
    \label{linking_Q}
\end{equation}

In [25]:
Q_linking_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(-solver.infinity(), 1)
        v_Q = Q[(j, k)]
        for i in I:
            v_X = X[(i, j, k)]

            c.SetCoefficient(v_X, 1)
            
        c.SetCoefficient(v_Q, -capacity)
        
        Q_linking_constraints.append(c)

assert len(Q_linking_constraints) == 438, "The number of constraints (" + str(len(Q_linking_constraints)) + ") is NOT correct"

#### 7.3 At least two vehicle types

\begin{equation}
    \sum_{i \in I} T_{i, j, k} \geq 2Q_{j, k}, \forall j \in J, k \in K
    \label{at_least_two_vehicle_types}
\end{equation}

In [26]:
atleast_two_types_of_vehicle_constraints = []

for j in J:
    capacity = capacities[j]
    for k in K:
        c = solver.Constraint(0, solver.infinity())
        v_Q = Q[(j, k)]
        for i in I:
            v_T = T[(i, j, k)]

            c.SetCoefficient(v_T, 1)
        c.SetCoefficient(v_Q, -2)
        atleast_two_types_of_vehicle_constraints.append(c)

assert len(atleast_two_types_of_vehicle_constraints) == 438, "The number of constraints (" + str(len(atleast_two_types_of_vehicle_constraints)) + ") is NOT correct"

### 8. Vehicles transfer

\begin{equation}
    X_{i, a, 1} - X_{i, a, 2} = \sum_{j \in J} \left ( Y_{i, a, j} - Y_{i, j, a} \right ), \forall i \in I, a \in J
    \label{vehicle_transfer}
\end{equation}

In [27]:
vehicles_transfer_constraints = []

for i in I:
    for a in J:
        c = solver.Constraint(0, 0)

        for j in J:
            Y_aj = Y[(i, a, j)]
            Y_ja = Y[(i, j, a)]

            c.SetCoefficient(Y_aj, 1)
            c.SetCoefficient(Y_ja, -1)
        
        X_a1 = X[(i, a, 1)]
        X_a2 = X[(i, a, 2)]

        c.SetCoefficient(X_a1, -1)
        c.SetCoefficient(X_a2, 1)

        vehicles_transfer_constraints.append(c)

assert len(vehicles_transfer_constraints) == 1095, "The number of constraints (" + str(len(vehicles_transfer_constraints)) + ") is NOT correct"

### 9. Maximum transfer distance

\begin{equation}
    r_{a, b}Y_{i, a, b} \leq l_{i} Y_{i, a, b}, \forall i \in \left \{ 1, 2, 3, 4 \right \}, a \in J, b \in J
    \label{maximum_transfer_distance}
\end{equation}

In [28]:
max_transfer_distance_constraints = []
a_counter = -1

for a in J:
    for b in J:
        for i in ["engine", "ladder", "rescue", "squad"]:
        
            route_distance = df_distances_stations.loc[a, b]

            #type_distance_limit = serviceability_distances[i] if i in ["engine", "ladder"] else (serviceability_distances[i] / 2)
            type_distance_limit = serviceability_distances[i]

            c = solver.Constraint(-solver.infinity(), 0)
            
            v = Y[(i, a, b)]

            c.SetCoefficient(v, route_distance - type_distance_limit)

            max_transfer_distance_constraints.append(c)

assert len(max_transfer_distance_constraints) == 191844, "The number of constraints (" + str(len(max_transfer_distance_constraints)) + ") is NOT correct"

### Objective Function

\begin{equation}
    maxZ: \sum_{k \in K}\sum_{i \in I}\sum_{j \in J}X_{j, i, k} \left ( \sum_{n \in N}s_{i, j, n} importance_{j, i, k, n} \right )
    \label{objective_function}
\end{equation}

\begin{equation}
    importance_{j, i, k, n} = w_{p} p^{\varsigma}_{n} + w_{e} e^{\varsigma}_{i, k, n} - w_{d} d^{\varsigma}_{j, n} - v^{\varsigma}_{i}
    \label{importance_factor}
\end{equation}

\begin{equation}
    w_{p}=0.4 \\
    w_{e}=0.1 \\
    w_{d}=0.5
    \label{weights}
\end{equation}

In [29]:
from sklearn.preprocessing import MinMaxScaler
MM_scaler = MinMaxScaler()
distance_scaled = MM_scaler.fit_transform(distance)
df_distance_scaled = distance.copy()
df_distance_scaled[:] = distance_scaled
df_distance_scaled.head()

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
QN08,0.712966,0.694468,0.704228,0.707264,0.681702,0.661229,0.645215,0.637852,0.689578,0.678506,...,0.646414,0.649381,0.645223,0.654903,0.810685,0.807546,0.847927,0.872241,0.840141,0.663432
BX28,0.631054,0.599836,0.643624,0.688625,0.648207,0.612963,0.569798,0.538405,0.619114,0.599397,...,0.855937,0.859395,0.835534,0.841614,0.847833,0.758353,0.727975,0.709328,0.711457,0.862873
QN55,0.623278,0.563582,0.599072,0.618132,0.585069,0.568148,0.549079,0.547624,0.595334,0.585267,...,0.548713,0.554022,0.547603,0.559543,0.728941,0.724448,0.769168,0.788933,0.757351,0.570430
BK50,0.501529,0.493496,0.528346,0.550370,0.547323,0.531786,0.511528,0.560687,0.558523,0.577703,...,0.436172,0.444186,0.435160,0.449707,0.634785,0.628735,0.678453,0.692978,0.661992,0.463308
BX41,0.588020,0.555329,0.600535,0.645981,0.601837,0.568300,0.523666,0.494979,0.573894,0.554524,...,0.820951,0.825249,0.789806,0.796946,0.809545,0.719431,0.691083,0.670305,0.672679,0.821289


In [30]:
availabilities_list = np.array(list(availabilities.values())).reshape(-1, 1)
vehicle_availabilities_scaled = MM_scaler.fit_transform(availabilities_list)
vehicle_availabilities_scaled


array([[1.        ],
       [0.7244898 ],
       [0.        ],
       [0.03571429],
       [0.02040816]])

In [31]:
availabilities_scaled = {'engine': 1.0, 'ladder': 0.7244898, 'hazardous': 0.0, 'squad': 0.03571429, 'rescue':0.02040816}

In [32]:
def set_objective_function(wp, we, wd):
    objective = solver.Objective()
    objective.SetMaximization()


    for k in K:
        shift_str = 'first' if k == 1 else 'second'
        for i in I:
            for j in J:
                v = X[(i, j, k)]
                overall_importance = 0
                
                for n in N:
                    #computing the coefficient
                    area_importance = 0

                    s = 1 if serviceability[i].loc[n][j] else 0

                    if s == 1:  
                        current_n = neighborhood[neighborhood['nta'] == n]
                        column_name = f"{i}_needed_{shift_str}_shift"

                        p = current_n['population_density'].values[0]
                        e = current_n[column_name].values[0]
                        d = df_distance_scaled.loc[n][j]
                        q = availabilities_scaled[i]
                        
                        importance = (wp * p) + (we * e) - (wd * d) - q
                        area_importance += importance

                    overall_importance += area_importance
                objective.SetCoefficient(v, overall_importance)
    
    return objective

In [33]:
def solve_problem(wp, we, wd):
    deployments_first_shift = pd.DataFrame(0, index=I, columns=J)
    deployments_second_shift = pd.DataFrame(0, index=I, columns=J)

    objective = set_objective_function(wp, we, wd)
    result = solver.Solve()

    if result == solver.ABNORMAL :
        print("Execution finished by an error")
        
    elif result == solver.FEASIBLE :
        print("In the specified time limit the solver has found a feasible solution")
        for k in K:
            for i in I:
                for j in J:
                    v = X[(i,j, k)]
                    if v.SolutionValue()>0:
                        print(v, v.solution_value())
        print("The value for the objective function is", objective.Value())
            
    elif result == solver.INFEASIBLE :
        print("There is no feasible solution for the problem")
            
    elif result == solver.NOT_SOLVED :
        print("In the specified time limit the solver has not found any feasible solution")
            
    elif result == solver.OPTIMAL :
        print("In the specified time limit the solver has found a feasible solution")
        for k in K:
            for i in I:
                for j in J:
                    v = X[(i,j,k)]
                    if v.SolutionValue()>0:
                        if k == 1:
                            deployments_first_shift.loc[i, j] = v.solution_value()
                        else:
                            deployments_second_shift.loc[i, j] = v.solution_value()
                        print(v, v.solution_value())
        
        print("The optimal value for the objective function is", objective.Value())
        
    elif result == solver.UNBOUNDED :
        print("The solution is unbounded")
        
    else :
        print("Unknown error code")

    return deployments_first_shift, deployments_second_shift

In [ ]:
wp = 0.25
we = 0.25
wd = 0.50

deployments_first_shift, deployments_second_shift = solve_problem(wp, we, wd)

deployments_first_shift.to_csv("dataset/results/deployments_first_" + str(int(wp*100)) + "p_" + str(int(we*100)) + "e_" + str(int(wd*100)) + "d.csv")
deployments_second_shift.to_csv("dataset/results/deployments_second_" + str(int(wp*100)) + "p_" + str(int(we*100)) + "e_" + str(int(wd*100)) + "d.csv")

In [ ]:
wp = 0.45
we = 0.5
wd = 0.50

deployments_first_shift, deployments_second_shift = solve_problem(wp, we, wd)

deployments_first_shift.to_csv("dataset/results/deployments_first_" + str(int(wp*100)) + "p_" + str(int(we*100)) + "e_" + str(int(wd*100)) + "d.csv")
deployments_second_shift.to_csv("dataset/results/deployments_second_" + str(int(wp*100)) + "p_" + str(int(we*100)) + "e_" + str(int(wd*100)) + "d.csv")

In [35]:
#results1 = pd.read_csv("dataset/results/deployments_first_01p_045e_045d.csv", index_col=0)
results2 = pd.read_csv("dataset/results/deployments_first_40p_10e_50d.csv", index_col=0)
results2

,42 South Street,49 Beekman Street,100 Duane Street,14 N. Moore Street,75 Canal Street,25 Pitt Street,222 East 2 Street,340 East 14 Street,253 Lafayette Street,42 Great Jones Street,...,256 Hylan Blvd.,278 McClean Avenue,1850 Clove Road,1592 Richmond Road,256 Nelson Ave.,345 Annadale Road,1560 Drumgoole Road West,7219 Amboy Road,1100 Rossville Ave,305 Front Street
engine,1,1,2,1,1,2,1,0,1,1,...,1,1,1,1,1,1,1,1,1,1
ladder,1,0,2,0,1,1,1,1,1,1,...,1,1,2,0,1,0,1,1,0,0
rescue,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
squad,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
hazardous,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/located_firehouses.json")
json_response = json.loads(response.text)

map = folium.Map(location=[40.70, -73.94], zoom_start=10, width="50%", tiles="CartoDB positron")
vehicle_colors = {
    'engine': 'blue',
    'ladder': 'red',
    'rescue': 'green',
    'squad': 'orange',
    'hazardous': 'purple'
}

#Legend text
legend_html = '''
<div style="position: fixed; bottom: 50px; right: 50px; z-index:1000; font-size:14px;">
     <p><strong>Legend</strong></p>
     <p><i class="fa fa-square fa-1x" style="color:blue;"></i> Engine</p>
     <p><i class="fa fa-square fa-1x" style="color:red;"></i> Ladder</p>
     <p><i class="fa fa-square fa-1x" style="color:green;"></i> Rescue</p>
     <p><i class="fa fa-square fa-1x" style="color:orange;"></i> Squad</p>
     <p><i class="fa fa-square fa-1x" style="color:purple;"></i> Hazardous</p>
     <p><i class="fa fa-star fa-1x" style="color:black;"></i> +1 vehicle</p>
</div>
'''

legend = folium.Element(legend_html)
map.get_root().html.add_child(legend)

#Adding markers to the map for each station
for idx, polygon in enumerate(json_response["features"]):
    facility_address = polygon["properties"]["FacilityAddress"]
    coordinates = [polygon["geometry"]["coordinates"][1], polygon["geometry"]["coordinates"][0]]
    
    for vehicle_type in results2.index:
        #Check if the station has just one unit of vehicle i
        if results2.loc[vehicle_type, facility_address] > 0 and results2.loc[vehicle_type, facility_address]<=1:
            color = vehicle_colors[vehicle_type]
            marker = folium.Marker(coordinates, tooltip=f"<b>Location</b>: {facility_address}", icon=folium.Icon(color=color))
            marker.add_to(map)
            
        #Check if the station has more than one unit of vehicle i   
        elif results2.loc[vehicle_type, facility_address] > 1:
            color = vehicle_colors[vehicle_type]
            marker = folium.Marker(coordinates, tooltip=f"<b>Location</b>: {facility_address}", icon=folium.Icon(color=color, icon="star"))
            marker.add_to(map)
map